In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from feature_engine.datetime import DatetimeFeatures
from feature_engine.timeseries.forecasting import LagFeatures, WindowFeatures, ExpandingWindowFeatures

In [3]:
data = pd.read_parquet('d:/demand-forecast-SQGroup/data/sales_bya_v2.parquet')

In [4]:
data.head().T

,0,1,2,3,4
cid,10001,10001,10001,10001,10001
item_type_id,186,186,186,186,186
category,1,1,1,1,1
date,2020-01-01 00:00:00,2020-01-01 00:00:00,2020-01-01 00:00:00,2020-01-01 00:00:00,2020-01-01 00:00:00
outlet_id,1003,1003,1003,1003,1003
wire,1,1,1,1,1
rm,1.0,1.0,1.0,1.0,1.0
fy,17,17,17,17,17
base_size,6,6,6,6,6
client_id,3DHEHE9HHE9H5MAK,3DHEHE9HHE9H9H1L,3DHEHE9HHEHE3DHE,3DHEHE9HHEHE4N7O,3DHEHE9HHEHEHEAK


In [5]:
df = data.copy()
df = df[['date', 'net_price', 'qtym']]

In [6]:
timeseries = df.groupby(by=['date']).sum().reset_index()

In [7]:
timeseries

,date,net_price,qtym
0,2019-07-01,2.627232e+07,823941.875000
1,2019-07-02,1.510582e+07,528482.937500
2,2019-07-04,5.594980e+06,197854.296875
3,2019-07-06,9.242950e+06,335050.500000
4,2019-07-07,9.014580e+06,314290.968750
...,...,...,...
1144,2023-06-25,4.866168e+04,1648.000000
1145,2023-06-26,2.878786e+06,100510.070312
1146,2023-06-27,4.076503e+06,147443.796875
1147,2023-06-28,5.416796e+06,192013.406250


<center><h1>Tempoal Features</h1></center>

In [8]:
def add_tempoal_features(df: pd.DataFrame) -> bool:
    features_needed = [
        "month", "quarter", "quarter_start", "quarter_end", "year_start", "year_end",
        "week", "day_of_week", "day_of_month", "day_of_year", "weekend", "month_start",
        "month_end"        
    ]
    try:
        extractor = DatetimeFeatures(
            variables=None, features_to_extract=features_needed, drop_original=True, 
            missing_values='raise', dayfirst=False, yearfirst=False, 
            utc=None, format=None
        )
        features = extractor.fit_transform(df)
        
        ###
        for col in list(features.columns)[2:]:
            df[col] = features[col].astype('int32').values
        
        return True
    except Exception:
        return False
    

In [9]:
extractor = DatetimeFeatures(
            variables=None, features_to_extract=["month", "quarter", "quarter_start", "quarter_end", "year_start", "year_end"], drop_original=True, 
            missing_values='raise', dayfirst=False, yearfirst=False, 
            utc=None, format=None
        )
extractor.fit_transform(timeseries)

,net_price,qtym,date_month,date_quarter,date_quarter_start,date_quarter_end,date_year_start,date_year_end
0,2.627232e+07,823941.875000,7,3,1,0,0,0
1,1.510582e+07,528482.937500,7,3,0,0,0,0
2,5.594980e+06,197854.296875,7,3,0,0,0,0
3,9.242950e+06,335050.500000,7,3,0,0,0,0
4,9.014580e+06,314290.968750,7,3,0,0,0,0
...,...,...,...,...,...,...,...,...
1144,4.866168e+04,1648.000000,6,2,0,0,0,0
1145,2.878786e+06,100510.070312,6,2,0,0,0,0
1146,4.076503e+06,147443.796875,6,2,0,0,0,0
1147,5.416796e+06,192013.406250,6,2,0,0,0,0


In [10]:
timeseries

,date,net_price,qtym
0,2019-07-01,2.627232e+07,823941.875000
1,2019-07-02,1.510582e+07,528482.937500
2,2019-07-04,5.594980e+06,197854.296875
3,2019-07-06,9.242950e+06,335050.500000
4,2019-07-07,9.014580e+06,314290.968750
...,...,...,...
1144,2023-06-25,4.866168e+04,1648.000000
1145,2023-06-26,2.878786e+06,100510.070312
1146,2023-06-27,4.076503e+06,147443.796875
1147,2023-06-28,5.416796e+06,192013.406250


In [11]:
add_tempoal_features(timeseries)

True

In [12]:
timeseries.head().T

,0,1,2,3,4
date,2019-07-01 00:00:00,2019-07-02 00:00:00,2019-07-04 00:00:00,2019-07-06 00:00:00,2019-07-07 00:00:00
net_price,26272324.0,15105817.0,5594980.5,9242950.0,9014580.0
qtym,823941.875,528482.9375,197854.296875,335050.5,314290.96875
date_month,7,7,7,7,7
date_quarter,3,3,3,3,3
date_quarter_start,1,0,0,0,0
date_quarter_end,0,0,0,0,0
date_year_start,0,0,0,0,0
date_year_end,0,0,0,0,0
date_week,27,27,27,27,27


In [13]:
timeseries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149 entries, 0 to 1148
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                1149 non-null   datetime64[ns]
 1   net_price           1149 non-null   float32       
 2   qtym                1149 non-null   float32       
 3   date_month          1149 non-null   int32         
 4   date_quarter        1149 non-null   int32         
 5   date_quarter_start  1149 non-null   int32         
 6   date_quarter_end    1149 non-null   int32         
 7   date_year_start     1149 non-null   int32         
 8   date_year_end       1149 non-null   int32         
 9   date_week           1149 non-null   int32         
 10  date_day_of_week    1149 non-null   int32         
 11  date_day_of_month   1149 non-null   int32         
 12  date_day_of_year    1149 non-null   int32         
 13  date_weekend        1149 non-null   int32       

<center><h1>Lag Features</h1></center>

In [14]:
def add_lag_features(df: pd.DataFrame) -> bool:
    
    try:
        lag = LagFeatures(
            variables=None, 
            periods=[1, 3, 7, 14, 28], freq=None, sort_index=True, 
            missing_values='raise', drop_original=False
        )
        
        features = lag.fit_transform(df[['date','net_price', 'qtym']])
        
        ###
        for col in list(features.columns)[2:]:
            df[col] = features[col].values
        
        return True
    except Exception:
        return False
    

In [15]:
lag = LagFeatures(variables=None, periods=[1, 3, 7, 14, 28], freq=None, sort_index=True, missing_values='raise', drop_original=False)
lag.fit_transform(timeseries[['date','net_price', 'qtym']]).head().T

,0,1,2,3,4
date,2019-07-01 00:00:00,2019-07-02 00:00:00,2019-07-04 00:00:00,2019-07-06 00:00:00,2019-07-07 00:00:00
net_price,26272324.0,15105817.0,5594980.5,9242950.0,9014580.0
qtym,823941.875,528482.9375,197854.296875,335050.5,314290.96875
net_price_lag_1,NaN,26272324.0,15105817.0,5594980.5,9242950.0
qtym_lag_1,NaN,823941.875,528482.9375,197854.296875,335050.5
net_price_lag_3,NaN,NaN,NaN,26272324.0,15105817.0
qtym_lag_3,NaN,NaN,NaN,823941.875,528482.9375
net_price_lag_7,NaN,NaN,NaN,NaN,NaN
qtym_lag_7,NaN,NaN,NaN,NaN,NaN
net_price_lag_14,NaN,NaN,NaN,NaN,NaN


In [16]:
add_lag_features(timeseries)

True

In [17]:
timeseries.head().T

,0,1,2,3,4
date,2019-07-01 00:00:00,2019-07-02 00:00:00,2019-07-04 00:00:00,2019-07-06 00:00:00,2019-07-07 00:00:00
net_price,26272324.0,15105817.0,5594980.5,9242950.0,9014580.0
qtym,823941.875,528482.9375,197854.296875,335050.5,314290.96875
date_month,7,7,7,7,7
date_quarter,3,3,3,3,3
date_quarter_start,1,0,0,0,0
date_quarter_end,0,0,0,0,0
date_year_start,0,0,0,0,0
date_year_end,0,0,0,0,0
date_week,27,27,27,27,27


<center><h1>Window Features</h1></center>

In [18]:
def add_window_features(df:pd.DataFrame) -> pd.DataFrame:
    window = WindowFeatures(
        variables=None, window=7, min_periods=7, 
        functions=['mean', 'std', 'median'], periods=7, freq=None, sort_index=True, 
        missing_values='raise', drop_original=False
    )
    features = window.fit_transform(df[['date','net_price', 'qtym']])
    for col in list(features.columns)[3:]:
        df[col] = features[col].values

In [19]:
add_window_features(timeseries)

In [20]:
timeseries.head(10).T

,0,1,2,3,4,5,6,7,8,9
date,2019-07-01 00:00:00,2019-07-02 00:00:00,2019-07-04 00:00:00,2019-07-06 00:00:00,2019-07-07 00:00:00,2019-07-09 00:00:00,2019-07-10 00:00:00,2019-07-11 00:00:00,2019-07-13 00:00:00,2019-07-15 00:00:00
net_price,26272324.0,15105817.0,5594980.5,9242950.0,9014580.0,10005522.0,6434367.5,6246905.0,9816375.0,10813265.0
qtym,823941.875,528482.9375,197854.296875,335050.5,314290.96875,330534.34375,241389.828125,232441.0625,352790.5625,364290.0
date_month,7,7,7,7,7,7,7,7,7,7
date_quarter,3,3,3,3,3,3,3,3,3,3
date_quarter_start,1,0,0,0,0,0,0,0,0,0
date_quarter_end,0,0,0,0,0,0,0,0,0,0
date_year_start,0,0,0,0,0,0,0,0,0,0
date_year_end,0,0,0,0,0,0,0,0,0,0
date_week,27,27,27,27,27,28,28,28,28,29


<center><h1>Expanding Features</h1></center>

In [21]:
def add_exp_window_features(df:pd.DataFrame) -> pd.DataFrame:
    expwindow = ExpandingWindowFeatures(
        variables=None, min_periods=7, functions='std', 
        periods=7, freq=None, sort_index=True, 
        missing_values='raise', drop_original=False
    )
    features = expwindow.fit_transform(df[['date', 'net_price', 'qtym']])
    
    ### 
    for col in list(features.columns)[3:]:
        df[col] = features[col].values

In [22]:
add_exp_window_features(timeseries)

In [23]:
timeseries.head().T

,0,1,2,3,4
date,2019-07-01 00:00:00,2019-07-02 00:00:00,2019-07-04 00:00:00,2019-07-06 00:00:00,2019-07-07 00:00:00
net_price,26272324.0,15105817.0,5594980.5,9242950.0,9014580.0
qtym,823941.875,528482.9375,197854.296875,335050.5,314290.96875
date_month,7,7,7,7,7
date_quarter,3,3,3,3,3
date_quarter_start,1,0,0,0,0
date_quarter_end,0,0,0,0,0
date_year_start,0,0,0,0,0
date_year_end,0,0,0,0,0
date_week,27,27,27,27,27


In [24]:
timeseries.dropna(axis=0, inplace=True)

In [25]:
timeseries.isnull().sum()

date                         0
net_price                    0
qtym                         0
date_month                   0
date_quarter                 0
date_quarter_start           0
date_quarter_end             0
date_year_start              0
date_year_end                0
date_week                    0
date_day_of_week             0
date_day_of_month            0
date_day_of_year             0
date_weekend                 0
date_month_start             0
date_month_end               0
net_price_lag_1              0
qtym_lag_1                   0
net_price_lag_3              0
qtym_lag_3                   0
net_price_lag_7              0
qtym_lag_7                   0
net_price_lag_14             0
qtym_lag_14                  0
net_price_lag_28             0
qtym_lag_28                  0
net_price_window_7_mean      0
net_price_window_7_std       0
net_price_window_7_median    0
qtym_window_7_mean           0
qtym_window_7_std            0
qtym_window_7_median         0
net_pric

In [26]:
timeseries.head().T

,28,29,30,31,32
date,2019-08-08 00:00:00,2019-08-10 00:00:00,2019-08-12 00:00:00,2019-08-13 00:00:00,2019-08-14 00:00:00
net_price,5042942.0,2932270.5,1809937.5,3067519.0,5123492.0
qtym,189950.125,130657.28125,79055.164062,122930.046875,211948.609375
date_month,8,8,8,8,8
date_quarter,3,3,3,3,3
date_quarter_start,0,0,0,0,0
date_quarter_end,0,0,0,0,0
date_year_start,0,0,0,0,0
date_year_end,0,0,0,0,0
date_week,32,32,33,33,33


In [27]:
timeseries.select_dtypes(['float64', 'float32']).columns[2:]

Index(['net_price_lag_1', 'qtym_lag_1', 'net_price_lag_3', 'qtym_lag_3',
       'net_price_lag_7', 'qtym_lag_7', 'net_price_lag_14', 'qtym_lag_14',
       'net_price_lag_28', 'qtym_lag_28', 'net_price_window_7_mean',
       'net_price_window_7_std', 'net_price_window_7_median',
       'qtym_window_7_mean', 'qtym_window_7_std', 'qtym_window_7_median',
       'net_price_expanding_std', 'qtym_expanding_std'],
      dtype='object')

<center><h1>Standardize Features</h1></center>

In [28]:
from sklearn.preprocessing import RobustScaler
import pickle

In [29]:

def standardize(df, save_scalers=False):

    ### Fetch Cols
    columns = df.select_dtypes(['float64', 'float32']).columns[2:]
    
    ### Standardizer
    scaler = RobustScaler(
            with_centering=True,
            with_scaling=True,
            quantile_range=(25.0, 75.0),
            copy=True,
            unit_variance=False,
        )
    
    ### Transform
    features = scaler.fit_transform(df[df.select_dtypes(['float64', 'float32']).columns[2:]])
    for index, column in enumerate(columns):
        df[column] = features[:, index]
    
    ### Save Scaler
    if save_scalers:
        pickle.dump(
        transformers,
        'd:/demand-forecast-SQGroup/data/standardizer.pkl',
        protocol=None,
        fix_imports=True,
        buffer_callback=None,
    )
    
    


In [30]:
standardize(timeseries)

In [31]:
timeseries

,date,net_price,qtym,date_month,date_quarter,date_quarter_start,date_quarter_end,date_year_start,date_year_end,date_week,...,net_price_lag_28,qtym_lag_28,net_price_window_7_mean,net_price_window_7_std,net_price_window_7_median,qtym_window_7_mean,qtym_window_7_std,qtym_window_7_median,net_price_expanding_std,qtym_expanding_std
28,2019-08-08,5.042942e+06,189950.125000,8,3,0,0,0,0,32,...,5.909094,5.015195,0.330076,1.038393,0.160170,0.190125,0.929874,0.165796,9.075584,5.551835
29,2019-08-10,2.932270e+06,130657.281250,8,3,0,0,0,0,32,...,2.914794,2.805540,0.298886,0.997825,0.160170,0.295423,1.082773,0.165796,8.633566,5.435308
30,2019-08-12,1.809938e+06,79055.164062,8,3,0,0,0,0,33,...,0.364461,0.332860,0.608128,0.757767,0.269587,0.673845,0.824075,0.576892,8.333165,5.133713
31,2019-08-13,3.067519e+06,122930.046875,8,3,0,0,0,0,33,...,1.342665,1.358912,0.412763,0.962069,0.269587,0.478127,1.035846,0.576892,8.958220,5.736917
32,2019-08-14,5.123492e+06,211948.609375,8,3,0,0,0,0,33,...,1.281428,1.203657,0.278132,1.019154,0.269587,0.370635,1.079417,0.576892,8.931170,5.683419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1144,2023-06-25,4.866168e+04,1648.000000,6,2,0,0,0,0,25,...,0.846822,0.673197,0.274822,0.531430,0.671697,0.232864,0.522264,0.751872,2.030117,0.836114
1145,2023-06-26,2.878786e+06,100510.070312,6,2,0,0,0,0,26,...,0.117538,0.100972,0.140761,0.545029,0.273397,0.077298,0.535589,0.319096,2.023339,0.830943
1146,2023-06-27,4.076503e+06,147443.796875,6,2,0,0,0,0,26,...,2.004125,1.736289,0.460900,0.272421,0.374126,0.367869,0.267568,0.319096,2.015974,0.825158
1147,2023-06-28,5.416796e+06,192013.406250,6,2,0,0,0,0,26,...,-1.128543,-1.141004,0.596299,0.278092,1.189326,0.517268,0.291428,1.359700,2.010849,0.821971


In [32]:
timeseries.to_parquet('d:/demand-forecast-SQGroup/data/sales_BYA_v3.parquet', index=False)